In [28]:
import numpy as np
import pandas as pd
import pprint

In [2]:
def get_pos_list(start, n_step, step_size):
    pos_list = []
    for i in range(n_step):
        pos_list.append(round((start + i * step_size), 4))
    return pos_list

In [23]:
motor1 = "CG1D:Mot:ShortAxis"
motor2 = "CG1D:Mot:LongAxis"
motor3 = "CG1D:Mot:G0:X"
# motor2 = "CG1D:Mot:G0:Roll"
# motor2 = None
# motor3 = "CG1D:Mot:G1:Roll"
# motor3 = "CG1D:Mot:ShortAxis"
# motor3 = None

fname = "G0_stepping"
delay = "0"
acq_time = 30
duplicate = 3

motor1_pos = [96, 30]
motor2_pos = [350, 118, 25]
motor3_pos = get_pos_list(start=92.4, n_step=12, step_size=0.1)
# motor2_pos = get_pos_list(start=5, n_step=3, step_size=0.1)
# motor3_pos = get_pos_list(start=20, n_step=2, step_size=0.1)

In [24]:
motor3_pos = sorted(motor2_pos*duplicate)

In [25]:
print(motor1_pos)
print(motor2_pos)
print(motor3_pos)

[96, 30]
[350, 118, 25]
[25, 25, 25, 118, 118, 118, 350, 350, 350]


In [26]:
output_file_name = fname+".csv"
fname_PV = "CG1D:Det:A1:TIFF1:FileName"
camera = "CCD 600"
header_list = ["CG1D:Det:A1:TIFF1:FileName"]
if motor1 is not None:
    header_list.append(motor1)
if motor2 is not None:
    header_list.append(motor2)
if motor3 is not None:
    header_list.append(motor3)
header_list.append("CG1D:Det:A1:AcquireTime")
header_list.append("Delay")
header_list.append("Camera")

In [33]:
scan_dict = {}
_motor1_pos_list = []
_motor2_pos_list = []
_motor3_pos_list = []
# _fname_list = []
if motor1 is not None:
    for pos1 in motor1_pos:
        if motor2 is None:
            _motor1_pos_list.append(pos1)
            _motor2_pos_list.append(None)
            _motor3_pos_list.append(None)
#             _fname_list.append(fname+"_"+str(pos1))
        else:
            _motor1_pos_list.append(pos1)
            if motor3 is None:
                for pos2 in motor2_pos:
#                     _motor1_pos_list.append(None)
                    _motor2_pos_list.append(pos2)
#                     _fname_list.append(fname+"_"+str(pos1)+"_"+str(pos2))
                _motor1_pos_list = _motor1_pos_list + (['']*(len(motor2_pos)-1))
            else:
                for pos2 in motor2_pos:
                    _motor2_pos_list.append(pos2)
                    for pos3 in motor3_pos:
                        _motor1_pos_list.append('')
                        _motor3_pos_list.append(pos3)
#                         _fname_list.append(fname+"_"+str(pos1)+"_"+str(pos2)+"_"+str(pos3))
                    _motor2_pos_list = _motor2_pos_list + (['']*(len(motor3_pos)-1))
                _motor1_pos_list = _motor1_pos_list + (['']*(len(motor2_pos) + len(motor3_pos)))
# scan_dict[fname_PV] = _fname_list
if motor1 is not None:
    scan_dict[motor1] = _motor1_pos_list[:]
if motor2 is not None:
    scan_dict[motor2] = _motor2_pos_list[:]
if motor3 is not None:
    scan_dict[motor3] = _motor3_pos_list[:]

pprint.pprint(scan_dict)
print(len(scan_dict[motor1]))
print(len(scan_dict[motor2]))
print(len(scan_dict[motor3]))
row_num = len(scan_dict[fname_PV])
scan_dict['Delay'] = [delay]*row_num
scan_dict["CG1D:Det:A1:AcquireTime"] = [acq_time]*row_num
scan_dict['Camera'] = [camera]*row_num

scan_df = pd.DataFrame.from_dict(scan_dict)
scan_df
        

{'CG1D:Mot:G0:X': [25,
                   25,
                   25,
                   118,
                   118,
                   118,
                   350,
                   350,
                   350,
                   25,
                   25,
                   25,
                   118,
                   118,
                   118,
                   350,
                   350,
                   350,
                   25,
                   25,
                   25,
                   118,
                   118,
                   118,
                   350,
                   350,
                   350,
                   25,
                   25,
                   25,
                   118,
                   118,
                   118,
                   350,
                   350,
                   350,
                   25,
                   25,
                   25,
                   118,
                   118,
                   118,
       

KeyError: 'CG1D:Det:A1:TIFF1:FileName'

In [12]:
scan_df.to_csv(output_file_name, index=False)